In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

C:\Users\dougl\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dougl\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dougl\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dougl\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework

In [2]:
base = pd.read_csv('iec6100429.csv')

In [3]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe

In [4]:
x.head()

,Amostra 1,Amostra 2,Amostra 3,Amostra 4,Amostra 5,Amostra 6,Amostra 7,Amostra 8,Amostra 9,Amostra 10,...,Amostra 182,Amostra 183,Amostra 184,Amostra 185,Amostra 186,Amostra 187,Amostra 188,Amostra 189,Amostra 190,Amostra 191
0,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
1,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
2,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
4,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3


In [5]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape

(92,)

In [6]:
y = y.values.reshape(-1,1)
y.shape

(92, 1)

In [7]:
y = onehot.fit_transform(y).toarray()

In [8]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.5)

In [9]:
x.shape, x_teste.shape, y.shape, y_teste.shape

((92, 191), (46, 191), (92, 7), (46, 7))

In [10]:
neuronios_entrada = x.shape[1]
neuronios_entrada

191

In [11]:
neuronios_oculta = 160
neuronios_oculta

160

In [12]:
neuronios_saida = y.shape[1]
neuronios_saida

7

In [13]:
w = {'oculta': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta, neuronios_saida]))}
w

{'oculta': <tf.Variable 'Variable:0' shape=(191, 160) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_1:0' shape=(160, 7) dtype=float32_ref>}

In [14]:
b = {'oculta': tf.Variable(tf.random_normal([neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}
b

{'oculta': <tf.Variable 'Variable_2:0' shape=(160,) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_3:0' shape=(7,) dtype=float32_ref>}

In [15]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [16]:
def mlp(x, w, bias):
    camada_oculta = tf.add(tf.matmul(x, w['oculta']), b['oculta'])
    camada_oculta_ativacao = tf.nn.relu(camada_oculta)
    camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, w['saida']), b['saida'])
    return camada_saida

In [17]:
modelo = mlp(xph, w, b)

In [18]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(erro)

In [19]:
batch_size = 5
batch_total = int(len(x_treinamento) / batch_size)
batch_total, (len(x_treinamento))

(9, 46)

In [20]:
x_batches = np.array_split(x_treinamento, batch_total)

In [21]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(10000):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 500 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])
    
    previsoes = tf.nn.softmax(mlp(xph, w_final, b_final))
    previsoes_corretas = tf.equal(tf.argmax(previsoes, 1), tf.argmax(yph, 1))
    taxa_acerto = tf.reduce_mean(tf.cast(previsoes_corretas, tf.float32))
    print(sess.run(taxa_acerto, feed_dict = {xph: x_teste, yph: y_teste}))

Época: 1 erro: 17876.736360338
Época: 501 erro: 0.023478917165322046
Época: 1001 erro: 3.4272648689026632
Época: 1501 erro: 0.002556961738517
Época: 2001 erro: 0.002810544612455285
Época: 2501 erro: 0.0006163420989145783
Época: 3001 erro: 3.7912523718356773e-05
Época: 3501 erro: 2.800933670086023e-05
Época: 4001 erro: 1.5406534758868576e-05
Época: 4501 erro: 3.1912000382463804e-05
Época: 5001 erro: 1.834903570675629e-05
Época: 5501 erro: 1.152565078478397e-05
Época: 6001 erro: 8.431813613343466e-06
Época: 6501 erro: 5.393469633771828e-06
Época: 7001 erro: 1.0362447562152336e-05
Época: 7501 erro: 1.3828192753824926e-06
Época: 8001 erro: 9.536722574113002e-07
Época: 8501 erro: 7.523417638013699e-07
Época: 9001 erro: 6.649217851872891e-07
Época: 9501 erro: 5.380226689845182e-06
0.95652175


In [22]:
previsoes_teste = mlp(xph, w_final, b_final)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(previsoes_teste, feed_dict = {xph: x_teste})
    r2 = sess.run(tf.nn.softmax(r1))
    r3 = sess.run(tf.argmax(r2, 1))
    
y_teste2 = np.argmax(y_teste, 1)
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(y_teste2, r3)
taxa_acerto

0.9130434782608695